### Wesley Janson and Drew Keller
## STAT 27420 Final Project
# Modeling Code

In [1]:
# Load in relevant packages

import pandas as pd
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from data_utils import read_data, prep_features, evaluate_predictions
import numpy as np
import matplotlib.pyplot as plt

# set random seed for numpy
RANDOM_SEED=69
np.random.seed(RANDOM_SEED)


DATA_PATH = '../paper_replication_data/new_data.csv'  # Drew's path

In [2]:
from load_data import data, categorical_vars, cts_vars, other_vars

# loading data from online takes ~20 seconds
# to speed up, save data locally and load from there:

#data.to_csv(DATA_PATH,index=False)  # run this once

Excluding 2262 observations that did not answer 1 year price change question.


In [3]:
# Categorical_vars and cts_vars are lists of vars in each category.
# Other_vars are ID and date variables (categorical_vars + cts_vars + other_vars = all vars)

data = read_data(DATA_PATH)  # use this over pd.read_csv, because this handles types

In [4]:
data.treatment_bins.value_counts(dropna=False)  # check that we have a balanced dataset

0-5      210475
5-10      47431
NaN       24168
10-15     11780
20+        5376
15-20      4984
Name: treatment_bins, dtype: int64

In [5]:
data.durable_purchase.value_counts(dropna=False)  # check that we have a balanced dataset

Good          204553
Bad            71471
Neutral        12945
Don't know     12599
Refused         2646
Name: durable_purchase, dtype: int64

***Regression***

In [4]:
# prep features for modeling; use regression=True for regression models
data_regression, treatment_vars, confounder_vars = prep_features(data,regression=True,missing_values='drop')  
X = data_regression[confounder_vars+["price_change_amt_next_yr"]]
Y = data_regression['durable_purchase']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)

Excluding 15245 observations that refused or didn't know durable purchase question.
Excluding 102089 observations that did not answer confounder questions.
Excluding 10354 observations that did not answer price change amount question.


In [21]:
# Model 1a: ordered probit (same as Bachmann et al.) with cts treatment
mod_prob1a = OrderedModel(y_train,X_train,distr='probit')
res_prob1a = mod_prob1a.fit(method='bfgs')
res_prob1a.summary()

Optimization terminated successfully.
         Current function value: 0.643999
         Iterations: 142
         Function evaluations: 143
         Gradient evaluations: 143


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:       durable_purchase   Log-Likelihood:                -90946.
Model:                   OrderedModel   AIC:                         1.820e+05
Method:            Maximum Likelihood   BIC:                         1.823e+05
Date:                Sat, 03 Dec 2022                                         
Time:                        19:41:48                                         
No. Observations:              141220                                         
Df Residuals:                  141186                                         
Df Model:                          34                                         
========================================================================================================
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
fed_funds_rate                           0.2261      0.007     34.525      0.000       0.213       0.239
unemployment_rate                       -0.1477      0.005    -32.673      0.000      -0.157      -0.139
cpi_1mo_lag                             -0.1669      0.007    -24.916      0.000      -0.180      -0.154
cpi_durable_1mo_lag                     -0.1476      0.006    -25.611      0.000      -0.159      -0.136
personal_finances_next_yr_Don't know    -0.1829      0.029     -6.303      0.000      -0.240      -0.126
personal_finances_next_yr_Refused       -0.2138      0.069     -3.111      0.002      -0.348      -0.079
personal_finances_next_yr_Same          -0.0482      0.008     -5.821      0.000      -0.064      -0.032
personal_finances_next_yr_Worse         -0.1618      0.013    -12.489      0.000      -0.187      -0.136
income_change_amt_next_yr               -0.0277      0.004     -7.104      0.000      -0.035      -0.020
conditions_next_yr_Don't know           -0.1279      0.036     -3.552      0.000      -0.198      -0.057
conditions_next_yr_Refused              -0.1049      0.070     -1.506      0.132      -0.241       0.032
conditions_next_yr_Same                 -0.0309      0.009     -3.394      0.001      -0.049      -0.013
conditions_next_yr_Worse                -0.2289      0.012    -19.427      0.000      -0.252      -0.206
unemployment_next_yr_Higher             -0.2251      0.062     -3.615      0.000      -0.347      -0.103
unemployment_next_yr_Lower               0.0673      0.063      1.076      0.282      -0.055       0.190
unemployment_next_yr_Refused             0.0502      0.123      0.407      0.684      -0.192       0.292
unemployment_next_yr_Same                0.0218      0.062      0.351      0.726      -0.100       0.144
income_quintile_Lower middle            -0.1070      0.012     -8.830      0.000      -0.131      -0.083
income_quintile_Lowest                  -0.2043      0.014    -14.823      0.000      -0.231      -0.177
income_quintile_Middle                  -0.0777      0.011     -6.877      0.000      -0.100      -0.056
income_quintile_Upper middle            -0.0515      0.010     -4.915      0.000      -0.072      -0.031
age                                     -0.0064      0.004     -1.518      0.129      -0.015       0.002
sex_Male                                 0.1101      0.007     14.740      0.000       0.095       0.125
education_Graduate school               -0.0210      0.012     -1.787      0.074      -0.044       0.002
education_High school                    0.0295      0.011      2.754      0.006       0.009       0.051
education_No high school                -0.1461      0.027     -5.471      0.000      -0.198      -0.094
education_Partial high school           -0.0188      0.019     -0.987      0.324      -0.056       0.019
education_So

In [22]:
evaluate_predictions(res_prob1a, X_train, X_test, y_train, y_test, regression=True)

Baseline accuracy: 73.73%
Train accuracy: 73.79%
Test accuracy: 74.13%

Test predictions vs actual:


actual  predicted
-1.0    -1             978
         1            7015
 0.0    -1              84
         1            1199
 1.0    -1             836
         1           25194
dtype: int64

In [9]:
X = data_regression[confounder_vars+treatment_vars]
Y = data_regression['durable_purchase']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)

In [10]:
# Model 1b: ordered probit (same as Bachmann et al.) with cts treatment
mod_prob1b = OrderedModel(y_train,X_train,distr='probit')
res_prob1b = mod_prob1b.fit(method='bfgs')
res_prob1b.summary()

Optimization terminated successfully.
         Current function value: 0.643949
         Iterations: 153
         Function evaluations: 154
         Gradient evaluations: 154


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:       durable_purchase   Log-Likelihood:                -90938.
Model:                   OrderedModel   AIC:                         1.820e+05
Method:            Maximum Likelihood   BIC:                         1.823e+05
Date:                Sat, 03 Dec 2022                                         
Time:                        19:29:01                                         
No. Observations:              141220                                         
Df Residuals:                  141182                                         
Df Model:                          38                                         
========================================================================================================
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
fed_funds_rate                           0.2265      0.007     34.597      0.000       0.214       0.239
unemployment_rate                       -0.1470      0.005    -32.509      0.000      -0.156      -0.138
cpi_1mo_lag                             -0.1652      0.007    -24.577      0.000      -0.178      -0.152
cpi_durable_1mo_lag                     -0.1478      0.006    -25.638      0.000      -0.159      -0.137
personal_finances_next_yr_Don't know    -0.1825      0.029     -6.288      0.000      -0.239      -0.126
personal_finances_next_yr_Refused       -0.2161      0.069     -3.147      0.002      -0.351      -0.082
personal_finances_next_yr_Same          -0.0487      0.008     -5.875      0.000      -0.065      -0.032
personal_finances_next_yr_Worse         -0.1621      0.013    -12.516      0.000      -0.188      -0.137
income_change_amt_next_yr               -0.0277      0.004     -7.121      0.000      -0.035      -0.020
conditions_next_yr_Don't know           -0.1279      0.036     -3.551      0.000      -0.198      -0.057
conditions_next_yr_Refused              -0.1047      0.070     -1.502      0.133      -0.241       0.032
conditions_next_yr_Same                 -0.0312      0.009     -3.432      0.001      -0.049      -0.013
conditions_next_yr_Worse                -0.2287      0.012    -19.413      0.000      -0.252      -0.206
unemployment_next_yr_Higher             -0.2237      0.062     -3.591      0.000      -0.346      -0.102
unemployment_next_yr_Lower               0.0683      0.063      1.093      0.274      -0.054       0.191
unemployment_next_yr_Refused             0.0515      0.123      0.417      0.677      -0.190       0.293
unemployment_next_yr_Same                0.0225      0.062      0.362      0.717      -0.099       0.144
income_quintile_Lower middle            -0.1072      0.012     -8.845      0.000      -0.131      -0.083
income_quintile_Lowest                  -0.2039      0.014    -14.800      0.000      -0.231      -0.177
income_quintile_Middle                  -0.0778      0.011     -6.886      0.000      -0.100      -0.056
income_quintile_Upper middle            -0.0516      0.010     -4.929      0.000      -0.072      -0.031
age                                     -0.0061      0.004     -1.442      0.149      -0.014       0.002
sex_Male                                 0.1112      0.007     14.878      0.000       0.097       0.126
education_Graduate school               -0.0210      0.012     -1.784      0.074      -0.044       0.002
education_High school                    0.0294      0.011      2.736      0.006       0.008       0.050
education_No high school                -0.1480      0.027     -5.541      0.000      -0.200      -0.096
education_Partial high school           -0.0209      0.019     -1.097      0.272      -0.058       0.016
education_So

In [18]:
evaluate_predictions(res_prob1b, X_train, X_test, y_train, y_test, regression=True)

Baseline accuracy: 73.73%
Train accuracy: 73.80%
Test accuracy: 74.08%

Test predictions vs actual:


actual  predicted
-1.0    -1             971
         1            7022
 0.0    -1              86
         1            1197
 1.0    -1             848
         1           25182
dtype: int64

***Classification with XGBoost***

In [3]:
data_xgboost, treatment_vars, confounder_vars = prep_features(data,regression=False,missing_values='retain')  

Excluding 12029 observations that refused or didn't know durable purchase question.
Excluding 16763 observations that did not answer price change amount question.


In [4]:
# only income_change_amt_next_yr has many missing values within our chosen cts confounders

# calculate percent missing for income_change_amt_next_yr by year 
pct_missing = data['income_change_amt_next_yr'].isnull().groupby(data.date.dt.year).agg(['mean','count'])
plt.plot(pct_missing["mean"])


AttributeError: Can only use .dt accessor with datetimelike values

In [5]:
data_xgboost, treatment_vars, confounder_vars = prep_features(data,regression=False,missing_values='impute by median')  

Excluding 12029 observations that refused or didn't know durable purchase question.
Imputing 60221 missing values for income_change_amt_next_yr with median.
Imputing 1346 missing values for age with median.
Imputing 467 missing values for household_size with median.
Excluding 16763 observations that did not answer price change amount question.


In [8]:
# Model 2: XGBoost with binned treatment

X = data_xgboost[confounder_vars+treatment_vars]
Y = data_xgboost['durable_purchase']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)
model2 = xgb.XGBClassifier()
model2.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [9]:
evaluate_predictions(model2, X_train, X_test, y_train, y_test)

Baseline accuracy: 73.88%
Train accuracy: 76.86%
Test accuracy: 74.66%

Test predictions vs actual:


actual  predicted
0.0     0             1637
        1                3
        2             8140
1.0     0              114
        2             1521
2.0     0             1290
        1                5
        2            30989
dtype: int64

***Classification with NN***

In [ ]:
# model 3: large neural network just to see if we can overfit at least
model3 = MLPClassifier(hidden_layer_sizes=(100,100), max_iter=1000)
model3.fit(X_train, y_train)

In [35]:
num_params = sum([coef.shape[0] * coef.shape[1] for coef in model3.coefs_])
num_params += sum([intercept.shape[0] for intercept in model3.intercepts_])
num_params

15603

In [30]:
evaluate_predictions(model3, X_train, X_test, y_train, y_test,regression=False)

Baseline accuracy: 71.63%
Train accuracy: 76.52%
Test accuracy: 70.52%

Test predictions vs actual:


actual  predicted
0.0     0             3076
        1              109
        2             9738
1.0     0              307
        1               30
        2             1916
2.0     0             3495
        1              206
        2            34619
dtype: int64

## Helper Functions

In [10]:
# Conditional outcome models (Q models)
def make_Q_model():
    ''' A function that returns a general ML q model for later use in k-folding'''
    return xgb.XGBRegressor()

# Propensity score models (g models)
def make_g_model():
    ''' A function that returns a g model for computing propensity scores'''
    return xgb.XGBClassifier()

In [ ]:
# Functions for K-fold cross-fitting
def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    '''
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    '''

    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
        X_train = X.loc[train_index]
        A_train = A.loc[train_index]
        g = make_model()
        g.fit(X_train, A_train)

        # get predictions for split
        predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]
    
    # sanity check that overlap holds
    assert np.isnan(predictions).sum() == 0
    return predictions

def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    '''
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"
    '''

    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1
    X2 = X_w_treatment.copy()
    X2["A"] = 2
    X3 = X_w_treatment.copy()
    X3["A"] = 3
    X4 = X_w_treatment.copy()
    X4["A"] = 4

    
    for train_index, test_index in kf.split(X_w_treatment, y):
        X_train = X_w_treatment.loc[train_index]
        y_train = y.loc[train_index]
        q = make_model()
        q.fit(X_train, y_train)

        if output_type =='binary':
            predictions0[test_index] = q.predict_proba(X0.loc[test_index])[:, 1]
            predictions1[test_index] = q.predict_proba(X1.loc[test_index])[:, 1]
        elif output_type == 'continuous':
            predictions0[test_index] = q.predict(X0.loc[test_index])
            predictions1[test_index] = q.predict(X1.loc[test_index])


    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [ ]:
# Fit single Q() model
# get conditional outcomes
Q0_lm, Q1_lm = outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, \
                                        n_splits=5, output_type="continuous")

# Fir 4 g(x) models
g1 = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=5)
g2 = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=5)
g3 = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=5)
g4 = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=5)